In [2]:
import pandas as pd
import numpy as np
import pygad
import random
import copy

In [3]:
import csv

# Initialize an empty dictionary
enginge_RUL = {}

with open('RUL_consultancy_predictions_A3-2.csv', newline='') as csvfile:
    # Create a CSV reader object with semicolon as delimiter
    reader = csv.reader(csvfile, delimiter=';')
    # Skip the header row
    next(reader)
    # Loop over each row in the CSV
    for row in reader:
        # Check if row has exactly 2 columns
        if len(row) == 2:
            enginge_RUL[int(row[1])] = int(row[0])

In [4]:
engine_RUL_30 = {key: value for key, value in enginge_RUL.items() if value <= 30}
engine_RUL_30

{20: 10,
 24: 26,
 31: 14,
 34: 8,
 35: 8,
 36: 24,
 37: 21,
 40: 29,
 41: 23,
 42: 13,
 49: 14,
 56: 18,
 61: 23,
 64: 24,
 66: 18,
 68: 13,
 76: 3,
 77: 27,
 81: 6,
 82: 11,
 90: 28,
 91: 29,
 92: 24,
 100: 24}

In [5]:
Team_A_time = {key: 4 for key in range(1, 21)}
Team_A_time.update({key: 3 for key in range(21, 56)})
Team_A_time.update({key: 2 for key in range(56, 81)})
Team_A_time.update({key: 8 for key in range(81, 101)})

Team_B_time = {j : Team_A_time[j] + 1 for j in range(1, 26)}
Team_B_time.update({j : Team_A_time[j] + 2 for j in range(26, 71)})
Team_B_time.update({j : Team_A_time[j] + 1 for j in range(71, 101)})

In [6]:
T1_time = Team_A_time
T2_time = Team_B_time
T3_time = Team_A_time
T4_time = Team_B_time

In [7]:
schedule = {"T1":[0,5], "T2":{}, "T3":{}, "T4":{}}
for i in schedule:
    print(i)

T1
T2
T3
T4


In [8]:
schedule["T1"]

[0, 5]

In [9]:
test = [0,2,4,5,8,6]
index = random.randrange(len(test))
selected_engine = test.pop(index)
print(index)
print(selected_engine)

1
2


In [10]:
test = 1
test += 2
test

3

In [11]:
def calculate_team_work_time(team, engines):
    if team in ["T1", "T3"]:
        engine_times = Team_A_time
    else:
        engine_times = Team_B_time
    
    time = 0
    
    for engine in engines:
        time += engine_times[engine]
    
    return time
    

def random_init(engines_RUL):
    schedule = {"T1":[], "T2":[], "T3":[], "T4":[]}
    
    possible_engines = list(engines_RUL.keys())
    possible_teams = ["T1", "T2", "T3", "T4"]
    
    iterations = 0 
    
    while possible_engines and iterations < 500:
        index = random.randrange(4)
        team = possible_teams[index]
        
        index = random.randrange(len(possible_engines))
        selected_engine = possible_engines[index]
        
        temp_schedule = copy.deepcopy(schedule)
        temp_schedule[team].append(selected_engine)
        
        if calculate_team_work_time(team, temp_schedule[team]) <= 30:
            schedule = temp_schedule
            possible_engines.remove(selected_engine)
        iterations += 1

    return schedule

In [12]:
random_init(engine_RUL_30)

{'T1': [35, 37, 64, 91, 76, 42, 81],
 'T2': [56, 34, 20, 100, 36],
 'T3': [77, 68, 90, 82, 66, 49, 40],
 'T4': [41, 31, 61, 92, 24]}

In [13]:
print("T1"+"_time")


T1_time


Loss Function

In [14]:
def engine_fix_date(schedule):
    engine_fix = {}
    for team in schedule:
        days = 0
        if team == "T1" or team == "T3":
            time_table = Team_A_time
        else:
            time_table = Team_B_time
        for engine in schedule[team]:
            engine_fix[engine] = days
            days = days + time_table[engine]
            
    return engine_fix


initial_schedule = random_init(engine_RUL_30)
print(initial_schedule)
engine_cost = {}

for engine in engine_RUL_30:
    engine_cost[engine] = 8
    


def loss_function(schedule, engine_RUL, engine_cost):
    loss = 0
    fix_dates = engine_fix_date(schedule)
    print(fix_dates)
    print(engine_RUL)
    for team in schedule:
        for engine in schedule[team]:

            days_late = fix_dates[engine] - engine_RUL[engine]
            engine_loss = 0
            if days_late > 0:
                cost = engine_cost[engine]
    
                for i in range(days_late):
                    day_loss = (cost * i)*(cost*i) # correct cost function needs to be inputted
                    if day_loss > 250:
                        day_loss = 250
                    engine_loss += day_loss
            loss += engine_loss
        
    return loss  

loss_schedule = loss_function(initial_schedule, engine_RUL_30, engine_cost)
print(loss_schedule)

{'T1': [20, 91, 40, 64, 66, 76, 36, 49, 34], 'T2': [100, 37, 31, 68, 56], 'T3': [24, 90, 82, 77, 92], 'T4': [41, 61, 42, 81, 35]}
{20: 0, 91: 4, 40: 12, 64: 15, 66: 17, 76: 19, 36: 21, 49: 24, 34: 27, 100: 0, 37: 9, 31: 14, 68: 19, 56: 23, 24: 0, 90: 3, 82: 11, 77: 19, 92: 21, 41: 0, 61: 5, 42: 9, 81: 14, 35: 23}
{20: 10, 24: 26, 31: 14, 34: 8, 35: 8, 36: 24, 37: 21, 40: 29, 41: 23, 42: 13, 49: 14, 56: 18, 61: 23, 64: 24, 66: 18, 68: 13, 76: 3, 77: 27, 81: 6, 82: 11, 90: 28, 91: 29, 92: 24, 100: 24}
16698
